In [1]:
import os
import json
from datetime import datetime
from bs4 import BeautifulSoup

In [2]:
print(datetime.now().strftime("%Y/%m/%d %H:%M:%S"))

2024/02/16 00:05:06


In [3]:
slozka = "filmy"

In [4]:
def seznam_souboru(slozka):
    seznam = []
    for file in os.listdir(slozka):
        seznam.append(file)
    return(seznam)

In [5]:
seznam_filmu = seznam_souboru(slozka)
seznam_filmu[0:10]

['10144-tabor-padlych-zen.html',
 '10173-eastern-krvavy-hugo.html',
 '102282-o-jecminkovi.html',
 '102314-zimni-vila.html',
 '10242-narkoblues.html',
 '103341-jan-hus-mse-za-tri-mrtve-muze.html',
 '103354-obcansky-prukaz.html',
 '103358-7-dni-hrichu.html',
 '103360-vojta-lavicka-nahoru-a-dolu.html',
 '103369-kuky-se-vraci.html']

## Začínáme scrapovat

### Nejdřív zjistíme, co všechno budeme scrapovat

In [6]:
kolik = None

(Čím menší číslo, tím menší vzorek a tím rychlejší test. Pro zpracování kompletního seznamu nutno změnit proměnou na None.)

In [7]:
def jake_jsou_sekce(slozka):
    sekce = []
    for s in seznam_souboru(slozka)[:kolik]:
        with open(os.path.join(slozka, s), "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")
            item_details = soup.find_all(class_='db-film-section')
            for item_detail in item_details:
                h2_tags = item_detail.find_all('h2')
                for h2_tag in h2_tags:
                    if h2_tag.get_text() not in sekce:
                        sekce.append(h2_tag.get_text())
    return(sekce)

In [8]:
sekce = jake_jsou_sekce("filmy")

In [9]:
sekce

['Anotace',
 'Obsah',
 'Hrají',
 'Štáb a tvůrci',
 'Hudba',
 'Lokace',
 'Produkční údaje',
 'Technické údaje',
 'Knihovna',
 'Poznámka',
 'Ocenění',
 'Galerie',
 'Komentář',
 'Písně',
 'Revue',
 'Účinkují',
 'Mluví',
 'Film online']

In [10]:
def jake_jsou_detaily(slozka):
    detaily = []
    for s in seznam_souboru(slozka)[:kolik]:
        with open(os.path.join(slozka, s), "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")
            item_details = soup.find_all(class_='item-details')
            for item_detail in item_details:
                h3_tags = item_detail.find_all('h3')
                for h3_tag in h3_tags:
                    if h3_tag.get_text() not in detaily:
                        detaily.append(h3_tag.get_text())
    return(detaily)

In [11]:
detaily = jake_jsou_detaily(slozka)

In [12]:
def jake_jsou_kredity(slozka):
    kredity = []
    for s in seznam_souboru(slozka)[:kolik]:
        with open(os.path.join(slozka, s), "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")
            item_details = soup.find_all(class_='db-film-credits-section')
            for item_detail in item_details:
                if "Štáb a tvůrci" in item_detail.text:
                    h3_tags = item_detail.find_all('h3')
                    for h3_tag in h3_tags:
                        if h3_tag.get_text() not in kredity:
                            kredity.append(h3_tag.get_text())
    return(kredity)

In [13]:
kredity = jake_jsou_kredity(slozka)

In [14]:
def jake_jsou_technicke_detaily(slozka):
    technika = []
    for s in seznam_souboru(slozka)[:kolik]:
        with open(os.path.join(slozka, s), "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")
            item_details = soup.find_all(class_='db-film-credits-section')
            for item_detail in item_details:
                if "Technické údaje" in item_detail.text:
                    h3_tags = item_detail.find_all('h3')
                    for h3_tag in h3_tags:
                        if h3_tag.get_text() not in technika:
                            technika.append(h3_tag.get_text())
    return(technika)

In [15]:
technika = jake_jsou_technicke_detaily(slozka)

In [16]:
technika

['Délka',
 'Minutáž',
 'Distribuční nosič',
 'Poměr stran',
 'Barva',
 'Zvuk',
 'Zvukový systém/formát',
 'Verze',
 'Mluveno',
 'Podtitulky',
 'Úvodní/závěrečné titulky',
 'Původní metráž',
 'Mezititulky',
 'Animační technika',
 'Natáčecí formát',
 'Rychlost']

## Scrapování filmů

In [17]:
def id_persony(url):
    if "cs/person/" in url:
        id_persony = url.split("/")[-2]
    else:
        id_persony = 0
    return(id_persony)

In [18]:
def pridej_atributy(slovnik, seznam, nalezene):
    for s in seznam:
        for n in nalezene:
            if s in n:
                n = n.replace(s," ")
                slovnik[s] = n.strip()

In [19]:
with open(os.path.join("data_fixes", "zeny_rucne.json"), encoding="utf-8") as zeny:
    zeny = json.loads(zeny.read())

In [20]:
def scrape_film(slozka, soubor):
    
    def jmeno_nejmeno(div):
        global zeny
        try:
            jmeno = id_persony(div.find("h3").find("a")["href"])   
        except:
            jmeno = div.find("h3").text
        if "/ž/" in div.find("h3").text:
            zena = div.find("h3").text.split("/ž/")[0].strip()
            if zena not in zeny:
                zeny.append(zena)
        return(jmeno)
            
    with open(os.path.join(slozka, soubor), "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
    
    atributy = {}
        
    atributy["Film"] = f"""{soup.find("h1").text.strip()} ({soup.find('meta', {'property': 'og:url'})['content'].split("/")[-2]})"""

    ## Detaily – země původu, copyright…
    
    item_details = soup.find_all(class_='item-details')
    divs_text = []

    for item_detail in item_details:
        divs = item_detail.find_all('div')
        for div in divs:
            obsah = div.text
            if div.find("span"):
                obsah = div.find("h3").text + ", ".join([s.text for s in div.find_all(class_="comma")])
            divs_text.append(obsah)

    pridej_atributy(atributy, detaily, divs_text)
    
    ## Credits
    
    item_details = soup.find_all(class_='db-film-credits-section')
    divs_text = []
    mluvi = []

    for item_detail in item_details:
        
        if "Hrají" in item_detail.find("h2").text:
            divs = item_detail.find_all(class_="credit-item")
            hraji = []
            postavy = []
            
            for div in divs:
                jmeno = jmeno_nejmeno(div) 
                hraji.append(jmeno)
                if div.find("p"):
                    role = div.find("p").text
                    if div.find("p").find("a"):
                        role = role.replace(div.find("p").find("a").text,"")
                        mluvi.append(id_persony(div.find("a")["href"]))
                    if div.find("p").find("span"):
                        role = role.replace(div.find("p").find("span").text,"")
                    postavy.append(role.strip())
            atributy["Hrají"] = hraji
            atributy["Postavy"] = postavy
            
        if "Mluví" in item_detail.find("h2").text:
            divs = item_detail.find_all(class_="credit-item")
            for div in divs:
                jmeno = jmeno_nejmeno(div)
                mluvi.append(jmeno)
            
        if mluvi:
            atributy["Mluví"] = mluvi
            
        if "Komentář" in item_detail.find("h2").text:
            komentar = []
            divs = item_detail.find_all(class_="credit-item")
            for div in divs:
                jmeno = jmeno_nejmeno(div)
                komentar.append(jmeno)
            atributy["Komentář"] = komentar

        if "Účinkují" in item_detail.find("h2").text:
            ucinkuji = []
            divs = item_detail.find_all(class_="credit-item")
            for div in divs:
                try:
                    jmeno = id_persony(div.find("a")["href"])
                except:
                    jmeno = div.find("h3").text 
                ucinkuji.append(jmeno)
            atributy["Účinkují"] = ucinkuji
            
        if "Štáb a tvůrci" in item_detail.find("h2").text:
            divs = item_detail.find_all(class_='credit-item')
            for div in divs:
                jmena_v_odkazech = []
                odkazy = div.find_all("a")
                for o in odkazy:
                    jmena_v_odkazech.append(o.text.strip())
                jmena_vsude = div.find("p").text.strip().split(",")
                jmena_vsude = [j.replace(", ","") for j in jmena_vsude if j]
                jmena_mimo_odkazy = [j.strip() for j in jmena_vsude if j.strip() not in jmena_v_odkazech]
                ids = [id_persony(o["href"]) for o in odkazy]
                if not ids:
                    ids = []
                if not jmena_mimo_odkazy:
                    jmena_mimo_odkazy = []
                divs_text.append(f"""{div.find("h3").text} {(", ".join(ids))}, {(", ").join(jmena_mimo_odkazy)}""")
                            
            pridej_atributy(atributy, kredity, divs_text)
            
        if "Technické údaje" in item_detail.find("h2").text or "Produkční údaje" in item_detail.find("h2").text:
            divs = item_detail.find_all('div')
            for div in divs:
                if div.find("h3"):
                    if div.find("p"):
                        if div.find("h3").text.strip() == "Zvuk":
                            atributy["Zvukový"] = div.find("p").text.strip()
                        else:
                            atributy[div.find("h3").text.strip()] = div.find("p").text.strip()
                        if "Žánr" in div.find("h3").text:
                            zanry = [z.strip() for z in div.find("p").text.split(",")]
                            zanry = ", ".join(zanry)
                            atributy[div.find("h3").text.strip()] = zanry
                        if div.find(class_="comma"):
                                atributy[div.find("h3").text.strip()] = ", ".join([s.text.strip() for s in div.find_all(class_="comma")])
                        if div.find("h3").text.strip() in ["Výrobce","S podporou","Nositelé copyrightu","Distribuce"]:
                            odkazy = div.find_all("a")
                            jmena_v_odkazech = [id_persony(o["href"]) for o in odkazy]
                            atributy[div.find("h3").text.strip()] = jmena_v_odkazech
                        
        if "Písně" in item_detail.find("h2").text:
            pisne = item_detail.find_all('h3')
            pisne = [pisen.text.strip() for pisen in pisne]
            atributy["Písně"] = pisne
            atributy["Hudba, text a zpěv písně"] = [id_persony(a["href"]) for a in item_detail.find_all("a")]
            
        if "Lokace" in item_detail.find("h2").text:
            lokace = item_detail.find('p')
            atributy["Lokace"] = lokace.text.strip()

        if "Ocenění" in item_detail.find("h2").text:
            oceneni = item_detail.find_all(class_="col-md-3")
            oceneni = [cena.text.strip() for cena in oceneni]
            atributy["Ocenění"] = oceneni
            
        if "Hudba" in item_detail.find("h2").text:
            divs = item_detail.find_all('div')
            for div in divs:
                atributy[div.find("h3").text] = [id_persony(a["href"]) for a in div.find_all("a")]
            
    item_details = soup.find_all(class_='film-description')
    
    for item_detail in item_details:
    
        if "Anotace" in item_detail.find("h2").text:
            atributy["Anotace"] = item_detail.find('p').text.strip()
            
        if "Obsah" in item_detail.find("h2").text:
            atributy["Obsah"] = item_detail.find('p').text.strip()
        
        if "Poznámka" in item_detail.find("h2").text:
            atributy["Poznámka"] = item_detail.find('p').text.strip()
        
    return(atributy)

## Test

In [21]:
scrape_film(slozka, "401929-skleneny-pokoj.html")

{'Film': 'Skleněný pokoj (401929)',
 'Země původu': 'Česká republika,                                                             Slovensko',
 'Copyright': '2019',
 'Rok výroby': '2018',
 'Premiéra': 'slavnostní premiéra 12. 3. 2019  (Univerzitní kino Scala, Brno)                                                                                                                                                                                                                                                        distribuční premiéra 14. 3. 2019 /nevhodné pro děti do 12 let/',
 'Minutáž': '105 min',
 'Režie': 'české verze 81624, 4528, Rudolf Biermann (supervize)',
 'Kategorie': 'film',
 'Žánr': 'historický, psychologický',
 'Typologie': 'hraný, distribuční',
 'Originální název': 'Skleněný pokoj',
 'Český název': 'Skleněný pokoj',
 'Anglický název': 'The Glass Room',
 'Souběžný název': 'Sklenená izba',
 'Hrají': ['Carice van Houten /ž/',
  'Hanna Alström /ž/',
  'Claes Bang',
  'Karel Roden',


In [22]:
scrape_film(slozka, "395251-manzele-pani-mileny.html")

{'Film': 'Manželé paní Mileny (395251)',
 'Země původu': 'Československo',
 'Copyright': '1921',
 'Rok výroby': '1921',
 'Premiéra': 'premiéra 28. 7. 1922 /nepřístupný mládeži/ (kino Beseda, Praha)',
 'Minutáž': '67 min',
 'Režie': '56169,',
 'Kategorie': 'film',
 'Žánr': 'drama',
 'Typologie': 'hraný, distribuční',
 'Originální název': 'Manželé paní Mileny',
 'Český název': 'Manželé paní Mileny',
 'Anglický název': 'The Husbands of Lady Milena',
 'Souběžný název': 'Pyrrhovo vítězství / Návrat mrtvého',
 'Stav filmu': 'nedochovaný',
 'Souběžný anglický název': 'The Pyrrhic Victory / Return of the Dead',
 'Hrají': ['127282',
  '104898',
  '125427',
  '127785',
  '125431',
  '60886',
  '68026',
  '4949',
  '125608',
  '55291',
  '83230',
  'Olga Purkrábková',
  '131464',
  'Klár',
  '127318'],
 'Postavy': ['paní Milena',
  'Milenin první muž malíř Pavel Jošt',
  'Milenin druhý muž dr. Jindřich Král',
  'Joštův přítel Hron',
  'impresario Eman Balcar',
  'Balcarova milenka',
  'Králova te

In [23]:
zeny

['Conny Dworak',
 'Carice van Houten',
 'Hanna Alström',
 'Alexandra Borbély',
 'Anouk Christiansen',
 'Tabitha Campbell',
 'Francesca Astorino']

## Konečně DataFrame

In [24]:
import pandas as pd

In [25]:
kolik = None

In [26]:
def frejm(slozka):
    df = pd.DataFrame()
    total = len(seznam_souboru(slozka)[:kolik])
    count = 0
    for s in seznam_souboru(slozka)[:kolik]:
        count = count + 1
        print(f"{count} / {total} / {s}")
        line = pd.DataFrame.from_dict([scrape_film(slozka, s)])
        df = pd.concat([df, line])
    df = df.reset_index(drop = True)
    return(df)

In [27]:
df = frejm("filmy")
df

1 / 4581 / 10144-tabor-padlych-zen.html
2 / 4581 / 10173-eastern-krvavy-hugo.html
3 / 4581 / 102282-o-jecminkovi.html
4 / 4581 / 102314-zimni-vila.html
5 / 4581 / 10242-narkoblues.html
6 / 4581 / 103341-jan-hus-mse-za-tri-mrtve-muze.html
7 / 4581 / 103354-obcansky-prukaz.html
8 / 4581 / 103358-7-dni-hrichu.html
9 / 4581 / 103360-vojta-lavicka-nahoru-a-dolu.html
10 / 4581 / 103369-kuky-se-vraci.html
11 / 4581 / 103372-habermannuv-mlyn.html
12 / 4581 / 103374-dobre-placena-prochazka.html
13 / 4581 / 103402-nebe-peklo.html
14 / 4581 / 103495-lidice.html
15 / 4581 / 103499-medvedi-ostrovy.html
16 / 4581 / 103509-dont-stop.html
17 / 4581 / 104213-az-na-veky.html
18 / 4581 / 10574-elfilm.html
19 / 4581 / 106663-dum.html
20 / 4581 / 106666-soukromy-vesmir.html
21 / 4581 / 106667-mamas-papas.html
22 / 4581 / 10781-karu-suda-medvedi-srdce.html
23 / 4581 / 10960-unos-domu.html
24 / 4581 / 1118-nachove-plachty.html
25 / 4581 / 11221-mechanika.html
26 / 4581 / 112328-jmenem-krale.html
27 / 4581 / 

205 / 4581 / 180001-vsechno-je-sracka.html
206 / 4581 / 180025-laska-v-hrobe.html
207 / 4581 / 180317-rekvalifikace.html
208 / 4581 / 180386-muzi-v-nadeji.html
209 / 4581 / 18042-ctyri-lasky.html
210 / 4581 / 18045-fimfarum-jana-wericha.html
211 / 4581 / 180492-bastardi-2.html
212 / 4581 / 180545-svata-ctverice.html
213 / 4581 / 18056-franta-nebojsa.html
214 / 4581 / 18058-splneny-sen.html
215 / 4581 / 180585-okresni-prebor-posledni-zapas-pepika-hnatka.html
216 / 4581 / 180618-probudim-se-vcera.html
217 / 4581 / 181233-ve-stinu-bohyne-matky-zeme.html
218 / 4581 / 181912-zneuzivany.html
219 / 4581 / 181928-skolni-vylet.html
220 / 4581 / 181956-co-chytnes-v-zite.html
221 / 4581 / 181960-osina.html
222 / 4581 / 181961-cesta.html
223 / 4581 / 181962-stoh.html
224 / 4581 / 18203-z-mesta-cesta.html
225 / 4581 / 182231-libas-jako-dabel.html
226 / 4581 / 18262-pupendo.html
227 / 4581 / 18264-jedna-ruka-netleska.html
228 / 4581 / 18265-utek-do-budina.html
229 / 4581 / 182826-laska-je-laska.html

412 / 4581 / 395109-dostavenicko-ve-mlynici.html
413 / 4581 / 395110-smich-a-plac.html
414 / 4581 / 395111-filmy-jana-krizeneckeho.html
415 / 4581 / 395111-vystavni-parkar-a-lepic-plakatu.html
416 / 4581 / 395112-to-nejlepsi-cislo.html
417 / 4581 / 395113-jarni-sen-stareho-mladence.html
418 / 4581 / 395114-hubicka.html
419 / 4581 / 395115-ponrepovo-kouzelnictvi.html
420 / 4581 / 395116-rudi-na-krtinach.html
421 / 4581 / 395117-rudi-na-zaletech.html
422 / 4581 / 395118-rudi-se-zeni.html
423 / 4581 / 395119-rudi-sportsman.html
424 / 4581 / 395120-sokove.html
425 / 4581 / 395121-ctyri-rocni-doby.html
426 / 4581 / 395122-dama-s-barzojem.html
427 / 4581 / 395123-falesny-hrac.html
428 / 4581 / 395124-pro-penize.html
429 / 4581 / 395125-zahadny-zlocin.html
430 / 4581 / 395126-zive-modely.html
431 / 4581 / 395127-americky-souboj.html
432 / 4581 / 395128-cholera-v-praze.html
433 / 4581 / 395129-estrella.html
434 / 4581 / 395130-faust.html
435 / 4581 / 395131-idyla-ze-stare-prahy.html
436 / 4581

615 / 4581 / 395312-mokre-dobrodruzstvi.html
616 / 4581 / 395313-on-a-ona.html
617 / 4581 / 395314-zkouska.html
618 / 4581 / 395315-bud-pripraven.html
619 / 4581 / 395316-carovne-oci.html
620 / 4581 / 395317-hrabenka-filangieri.html
621 / 4581 / 395318-kdyz-srdce-promluvi.html
622 / 4581 / 395319-madame-golvery.html
623 / 4581 / 395320-muz-bez-srdce.html
624 / 4581 / 395321-palicova-dcera.html
625 / 4581 / 395322-pepanek-nezdara.html
626 / 4581 / 395323-pro-a-proti.html
627 / 4581 / 395324-problematicky-gentleman.html
628 / 4581 / 395325-rasputin.html
629 / 4581 / 395326-tu-ten-kamen.html
630 / 4581 / 395327-unos-bankere-fuxe.html
631 / 4581 / 395328-za-oponou-smrti.html
632 / 4581 / 395329-zahadny-pripad-galginuv.html
633 / 4581 / 395330-zavet-podivinova.html
634 / 4581 / 395331-bily-raj.html
635 / 4581 / 395332-chytte-ho.html
636 / 4581 / 395333-dama-z-baru.html
637 / 4581 / 395334-devce-z-hor.html
638 / 4581 / 395335-dvoji-zivot.html
639 / 4581 / 395336-hrichy-v-manzelstvi.html
640 

812 / 4581 / 395510-operene-stiny.html
813 / 4581 / 395511-pili-a-pokrokem-k-uspechu.html
814 / 4581 / 395512-pradlenka-jeho-jasnosti.html
815 / 4581 / 395513-tajemstvi-lekarovo.html
816 / 4581 / 395514-utrpeni-sede-sestry.html
817 / 4581 / 395515-v-kropenate-lhote-hori.html
818 / 4581 / 395516-vendelinuv-ocistec-a-raj.html
819 / 4581 / 395517-vse-pro-lasku.html
820 / 4581 / 395518-za-rodnou-hroudu.html
821 / 4581 / 395519-rada-lekare.html
822 / 4581 / 395520-tonka-sibenice.html
823 / 4581 / 395521-film-se-sasou-rasilovem-a-karlem-faltysem.html
824 / 4581 / 395522-kdyz-struny-lkaji-nemecka-verze.html
825 / 4581 / 395523-on-a-jeho-sestra-nemecka-verze.html
826 / 4581 / 395524-afera-plukovnika-redla-nemecka-verze.html
827 / 4581 / 395525-c-a-k-polni-marsalek-francouzska-verze.html
828 / 4581 / 395526-zena-ktera-se-smeje.html
829 / 4581 / 395527-svet-bez-hranic.html
830 / 4581 / 395528-afera-plukovnika-redla.html
831 / 4581 / 395529-pana-bumlicka-novy-domov.html
832 / 4581 / 395530-pensio

...

3146 / 4581 / 398589-konec-drahoty-topeni-alfa-thermo.html
3147 / 4581 / 398590-letime-raketou.html
3148 / 4581 / 398591-nezapomenutelny-plakat.html
3149 / 4581 / 398592-pisen-podzimu.html
3150 / 4581 / 398593-strevicek.html
3151 / 4581 / 398594-smudlik-a-leskulinka.html
3152 / 4581 / 398595-vsechno-pro-trhanec.html
3153 / 4581 / 398596-zlata-praha.html
3154 / 4581 / 398597-velka-soutez.html
3155 / 4581 / 398598-jaro-je-tady.html
3156 / 4581 / 398599-kouzlo-slunce.html
3157 / 4581 / 398600-myslenka-hledajici-svetlo.html
3158 / 4581 / 398601-nakaza.html
3159 / 4581 / 398602-protest.html
3160 / 4581 / 398603-stehovani.html
3161 / 4581 / 398604-vestba-pani-antonie.html
3162 / 4581 / 398605-zapas-ve-vzduchu.html
3163 / 4581 / 398606-zkouska-v-ohni.html
3164 / 4581 / 398607-zadame-sve-pravo.html
3165 / 4581 / 398608-dobra-rada-nad-zlato.html
3166 / 4581 / 398609-technika-rozhlasu.html
3167 / 4581 / 398610-vcela.html
3168 / 4581 / 398611-prani-k-novemu-roku.html
3169 / 4581 / 398612-laska-a-

3340 / 4581 / 400421-vzpoura-trestance-sejby.html
3341 / 4581 / 400422-vzorna-skola-mocnarstvi.html
3342 / 4581 / 400423-vzorne-rodinne-stesti.html
3343 / 4581 / 400424-vzorny-mecenas.html
3344 / 4581 / 400425-vzorna-laska-lepsich-lidi.html
3345 / 4581 / 400426-o-chlapci-ktery-hledal-konec-sveta.html
3346 / 4581 / 400427-lide-na-zemi-a-hvezdy-na-nebi.html
3347 / 4581 / 400428-andela.html
3348 / 4581 / 400429-maminka.html
3349 / 4581 / 400430-mala-vanocni.html
3350 / 4581 / 400431-milenci-z-bedny.html
3351 / 4581 / 400432-mistr-a-dvacaty-ucednik.html
3352 / 4581 / 400433-kazdy-tyden-je-nedele.html
3353 / 4581 / 400434-to-zavinil-bonifac.html
3354 / 4581 / 400435-otrhana-pisnicka.html
3355 / 4581 / 400436-pavucina.html
3356 / 4581 / 400437-taxi-prosim.html
3357 / 4581 / 400438-dalky.html
3358 / 4581 / 400439-zaza.html
3359 / 4581 / 400440-kdyby-ty-muziky-nebyly.html
3360 / 4581 / 400441-konkurs.html
3361 / 4581 / 400442-manzel.html
3362 / 4581 / 400443-profesor.html
3363 / 4581 / 400444-

3538 / 4581 / 400665-mala-z-rybarny.html
3539 / 4581 / 400666-maly-pan.html
3540 / 4581 / 400667-kandidat.html
3541 / 4581 / 400668-velka-noc.html
3542 / 4581 / 400669-rosa-dara-a-jejich-velka-dobrodruzstvi.html
3543 / 4581 / 400670-rosa-dara-a-jejich-velke-prazdninove-dobrodruzstvi.html
3544 / 4581 / 400671-novy-druh.html
3545 / 4581 / 400672-rosa-dara-proc-prsi.html
3546 / 4581 / 400673-maly-cousteau.html
3547 / 4581 / 400674-rosa-dara-co-je-to-gravitace.html
3548 / 4581 / 400675-mythopolis.html
3549 / 4581 / 400676-rosa-dara-jak-velke-jsou-hvezdy.html
3550 / 4581 / 400677-opusteny-vesmir.html
3551 / 4581 / 400678-opri-zebrik-o-nebe.html
3552 / 4581 / 400679-muzikal-aneb-cesty-ke-stesti.html
3553 / 4581 / 400680-the-deal.html
3554 / 4581 / 400691-v-jinem-stavu.html
3555 / 4581 / 400694-dvojnici.html
3556 / 4581 / 400696-lida-baarova.html
3557 / 4581 / 400697-noc-bezmoci.html
3558 / 4581 / 400698-rudy-kapitan.html
3559 / 4581 / 400700-stastna.html
3560 / 4581 / 400701-rachanda.html
35

3735 / 4581 / 401679-camino-na-koleckach.html
3736 / 4581 / 401680-satanova-jizda-po-zeleznici.html
3737 / 4581 / 401681-pomnik-frantiska-palackeho-pred-dokoncenim.html
3738 / 4581 / 401682-oslava-usporadana-na-pocest-550leteho-c-k-privilegovaneho-sb.html
3739 / 4581 / 401683-iv-vsesokolsky-slet-v-praze.html
3740 / 4581 / 401684-slavnostni-vysveceni-mostu-cisare-frantiska-i.html
3741 / 4581 / 401685-cviceni-hasicu.html


TypeError: 'NoneType' object is not subscriptable

In [ ]:
df["Asistent vedoucího produkce"].drop_duplicates().to_list()

In [ ]:
df.columns

## Export

In [ ]:
with open(os.path.join("data_fixes", "zeny.json"), "w+", encoding="utf-8") as zeny_soubor:
    json.dump(zeny, zeny_soubor)

In [ ]:
df.to_json(os.path.join("data", "filmy_raw.json"), orient="records")

In [ ]:
print(datetime.now().strftime("%Y/%m/%d %H:%M:%S"))